In [ ]:
import pandas as pd
import json
import re
import yfinance as yf
import requests

In [ ]:
# reload the CSV file into a DataFrame
df = pd.read_csv('../data-2023/out/df_data_typed.csv')

# Reset the index to remove the 'Unnamed: 0' column from the DataFrame
df.reset_index(drop=True, inplace=True)

# Add Year attribute
year = 2023
df['Year']= 2023

In [ ]:
# filter only corporate bonds
df = df[df['Asset Type'] == 'Corporate Bonds']


In [ ]:
# sanity check uncleaned
for idx, name in enumerate(df['Name'], start=1):
    print(f"{idx}: {name}")

In [ ]:
# TRIM UNNECESSARY PREFIXES

# trim PVTPL, PFS, PASS, THRU, CMO, COML, MTG, SR, TR, SER, DUE, POOL, 
df['Name'].replace( { r"\b(PVPTL|PVTPL|PVTPL1|PVYPL|PFS|PVT|PL|ESC)\b" : '' }, inplace= True, regex = True)
# trim words with hyphens 
df['Name'].replace( { r'(\d+(\.\d+)?%)(.*)' : '' }, inplace= True, regex = True)
# trim unnec suffixes 
df['Name'].replace( { r"\b(INC|)\b" : '' }, inplace= True, regex = True)

In [ ]:
# here's them cleaned
for idx, name in enumerate(df['Name'], start=1):
    print(f"{idx}: {name}")

In [ ]:
# Add Tickers
df['Ticker'] = str('NaN')

In [ ]:
# WARNING: this only works for this dataset! the following tickers were hand-annotated and crosslisted with 
# the original investments
company_dict = {
    "VMWARE": "VMW",
    "ADVENTIST": "SDA",
    "AQUARION": "ES", # acquired by ES
    "ASCENSION": "ACE.NZ",
    "COMMONSPIRIT": "US:US20268JAA16",
    "DOWDUPONT": "DD", 
    "LACLEDE": "SPIR",
    "LULWA": "DISSOLVED",
    "ONCOR": "SRE", # majority owner
    "PEACEHEALTH": "US:US70462GAA67",
    "RABOBANK": "RABO.AS",
    "GULFSTREAM": "GD", # acquired by GD
    "PRICOA": "US:US74153XBH98",
    "AVOLON": "AVOL",
    "3.05%": "INVALID",
    "LSEGA": "LSEG.L", # london stock exchange?
    "GLAXOSMITHKLINE": "GSK",
    "INTERCONTINENTAL": "IHG",
    "JPMORGAN": "JPM",
    "KANSAS": "EVRG", # kansas city power now evergy
    "KLA": "KLAC", # kla corp
    "LLOYDS": "LYG",
    "NATWEST": "NWG",
    "NXP": "NXPI",
    "PEPSICO": "PEP",
    "BAE": "BAESY",
    "BMW": "BMWYY",
    "KIA": "KRX",
    "MACQUARIE": "MQG.AX",
    "PACIFIC": "0593430D:US", # not publically traded
    "VOLKSWAGEN": "VWAGY",
    "S&P": "SPGI",
    "TEXTRON": "TXT",
    "COMCAST": "CMCSA",
    "CES MU2 LLC": "???",
    "CHEVRON": "CVX",
    "CITIGROUP": "C",
    "CITIZENS BK": "CFG",
    "CONOCOPHILLIPS": "COP",
    "DEERE JOHN": "DE",
    "ECOLAB": "ECL",
    "EMORY": "US:US29157TAD81",
    "ENBRIDGE": "EMB",
    "ENTERGY": "ETR",
    "ERP": "???",
    "FEDERAL RLTY INVT": "FRT",
    "FISERV": "FI",
    "FMC CORP": "FMC",
    "GENERAL DYNAMICS": "GD",
    "GENERAL MLS": "GIS",
    "GENERAL MTRS": "GM",
    "GEORGIA-PAC": "???",
    "GETTY PAUL": "US374288AB41",
    "GLOBAL PMTS": "GPN",
    "HCP": "HCP",
    "HEWLETT PACKARD": "HPQ",
    "HYUNDAI": "HYMTF",
    "KEY BANK": "KEY",
    "MARTIN MARIETTA": "MLM",
    "MITSUBISHI": "MSBHF",
    "MONDELEZ INTERNATIONAL": "MDLZ",
    "NATL BK CDA MEDIUM": "NTIOF",
    "NOVARTIS": "NVS",
    "ONE GAS": "OGS",
    "PACCAR FINL": "PCAR",
    "PARKER-HANNIFIN": "PH",
    "SIMON PPTY": "SPG",
    "WEINGARTEN": "WP4.F",
    "AMERICAN HONDA": "HMC",
    "BP": "BP",
    "BPCE S A MEDIUM": "ENXTPA:BPCEGP",
    "CANADIAN IMPERIAL": "CM",
    "CARDINAL HEALTH": "CAH",
    "CATERPILLAR": "CAT",
    "COOPERATIEVE RABOBANK": "RABO.AS",
    "TRANS-CDA": "TRP",
    "BOSTON PPTYS": "BXP",
    "BRISTOL MYERS SQUIBB": "BMY",
    "CANADIAN NAT RES LTD": "CNQ",
    "CANADIAN PAC RY": "CP",
    "CAPITAL ONE": "COF",
    "CENTERPOINT ENERGY": "CNP",
    "CHARLES SCHWAB": "SCHW",
    "CNH INDL": "CNHI",
    "CONNECTICUT LIGHT & POWER": "CNTHO",
    "DAIMLER TRUCKS": "DTRUY",
    "DTE ELEC": "DTE",
    "DUKE ENERGY": "DUK",
    "DUKE RLTY": "DRE",
    "EATON": "ETN",
    "ENTERPRISE PRODS": "EPD",
    "ERP OPER": "EPD",
    "EXXON": "XOM",
    "FIFTH THIRD": "FITB",
    "GILEAD SCIENCES": "GILD"
}

In [ ]:
# # unpack company_dict from json file
# import json

# # Opening JSON file
# company_dict = open('../data-2023/in/company_dict.json')
 
# # returns JSON object as 
# # a dictionary
# company_dict = json.load(company_dict)

In [ ]:
# taken from https://gist.github.com/bruhbruhroblox/dd9d981c8c37983f61e423a45085e063
def getTicker(name):
    try: 
        yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        params = {"q": name, "quotes_count": 1, "country": "United States"}

        res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
        data = res.json()
        ticker = data['quotes'][0]['symbol']
        return ticker
    except: 
        print(name)

def getTickerWrapper(company_name):
    words = company_name.split()
#     print(words)
    for i in range(len(words), 0, -1):
        curr_name = ' '.join(words[:i])
#         print(curr_name)
        # check against company dict 
        if curr_name in company_dict:
            return company_dict[curr_name]
        ticker = getTicker(curr_name)
        if ticker:
            company_dict[curr_name] = ticker 
            return ticker 
    return "Error"

# How this works
The following three blocks are run iteratively in increments of 100 entries (i.e. df[:100], then df[101:200], etc.) for speed and human review. 

In [ ]:
# 1st round - grab tickers for all in increments of 300 (0-300, 301-600, etc.)
for index, row in df[:100].iterrows():
#     print(getTickerWrapper(row['Name']))
#     row['Ticker'] = getTickerWrapper(row['Name'])
    df.at[index, 'Ticker'] = getTickerWrapper(row['Name'])

In [ ]:
# second round - fill any companies with Not Found in company_dict
for index, row in df[:100].iterrows():
    if row['Ticker'] == "Error" and row['Ticker'] in company_dict:
        df.at[index, 'Ticker'] = company_dict[row['Ticker']]

In [ ]:
df[:100]

# End of Iterative Script
At this point, we are operating under the assumption that all increments have been run, i.e. the entire dataframe's tickers have been generated. Only run the next block once this is true, otherwise you will rewrite the entire existing dataframe CSV file and all unpopulated ticker values will be NaN.

In [ ]:
# At this point, inspect the file to see if there are any incorrect / erroneous tickers.
df.to_csv("../data-2023/out/df_with_ticker_0_to_100.csv")

# Graveyard: Do Not Run After this Point

In [ ]:
import json

# Convert and write JSON object to file
with open("company_dict.json", "w") as outfile: 
    json.dump(company_dict, outfile)

In [ ]:
# drop final error and ??? rows
df = df[df['Ticker'] != ]